In [1]:
import sys
import os
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, project_root)


import InterfaceWidgets.data_load_widgets as data_load_widgets

# Показываем интерфейс
data_load_widgets.display_interface()

Output()

Output()